# Auto-Fix

> Automatically add placeholder documentation to non-compliant functions

In [ ]:
#| default_exp autofix

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import ast
from typing import List, Dict, Any, Optional, NamedTuple
import re
from pathlib import Path
from execnb.nbio import read_nb, write_nb
from fastcore.foundation import L
from fastcore.basics import ifnone, patch, compose
from cjm_nbdev_docments.core import DocmentsCheckResult, check_definition
from cjm_nbdev_docments.scanner import scan_notebook, get_export_cells

In [ ]:
#| export
@patch
def needs_fixing(
    self: DocmentsCheckResult
) -> bool:  # TODO: Add return description
    "Check if this definition needs any fixing"
    return not self.is_compliant or self.missing_params or self.params_missing_type_hints

In [ ]:
#| export
@patch
def get_param_name(
    self: DocmentsCheckResult,
    param_str: str  # TODO: Add description
) -> str:  # TODO: Add return description
    "Extract parameter name from a parameter string"
    return param_str.split(':', 1)[0].split('=', 1)[0].strip()

In [ ]:
#| export
@patch 
def needs_param_fix(
    self: DocmentsCheckResult,
    param_name: str  # TODO: Add description
) -> bool:  # TODO: Add return description
    "Check if a parameter needs documentation or type hint fixes"
    needs_doc = param_name in self.missing_params and param_name != 'self'
    needs_type_hint = param_name in self.params_missing_type_hints and param_name != 'self'
    return needs_doc or needs_type_hint

In [ ]:
#| export
def find_signature_boundaries(
    lines: List[str]  # Source code lines
) -> tuple[int, int]:  # (def_line_idx, sig_end_idx) or (-1, -1) if not found
    "Find the start and end lines of a function signature"
    def_line_idx = None
    sig_end_idx = None
    paren_count = 0
    in_signature = False
    
    for i, line in enumerate(lines):
        if line.strip().startswith(('def ', 'async def ')):
            def_line_idx = i
            in_signature = True
            
        if in_signature:
            # Count parentheses to find where signature ends
            paren_count += line.count('(') - line.count(')')
            
            # If we're back to balanced parens and line contains a colon, signature is done
            # (colon might be followed by comments)
            if paren_count == 0 and ':' in line:
                sig_end_idx = i
                break
    
    # Use ifnone for cleaner null handling
    def_line_idx = ifnone(def_line_idx, -1)
    sig_end_idx = ifnone(sig_end_idx, -1)
    
    if def_line_idx == -1 or sig_end_idx == -1:
        return -1, -1
    
    return def_line_idx, sig_end_idx

In [ ]:
#| export
def split_parameters(
    params_str: str  # Parameter string from function signature
) -> List[str]:  # List of individual parameter strings
    "Split a parameter string into individual parameters, handling nested types"
    if not params_str.strip():
        return []
    
    # Use a more robust approach for complex nested types
    params = []
    current_param = ''
    paren_depth = 0
    bracket_depth = 0
    brace_depth = 0
    
    for char in params_str:
        if char == '(':
            paren_depth += 1
        elif char == ')':
            paren_depth -= 1
        elif char == '[':
            bracket_depth += 1
        elif char == ']':
            bracket_depth -= 1
        elif char == '{':
            brace_depth += 1
        elif char == '}':
            brace_depth -= 1
        elif char == ',' and paren_depth == 0 and bracket_depth == 0 and brace_depth == 0:
            params.append(current_param.strip())
            current_param = ''
            continue
        current_param += char
    
    if current_param.strip():
        params.append(current_param.strip())
    
    # Return as L for easier manipulation
    return L(params).filter()

In [ ]:
#| export
def parse_single_line_signature(
    sig_line: str  # Single-line function signature
) -> dict:  # Parsed components of the signature
    "Parse a single-line function signature into its components"
    func_match = re.match(r'^(\s*)(def|async def)\s+(\w+)\s*\((.*?)\)(\s*(?:->\s*[^:]+)?)\s*:\s*(.*)$', sig_line)
    if not func_match:
        return None
    
    return {
        'indent': func_match.group(1),
        'def_keyword': func_match.group(2),
        'func_name': func_match.group(3),
        'params_str': func_match.group(4),
        'return_type': func_match.group(5),
        'existing_comment': func_match.group(6).strip()
    }

In [ ]:
#| export
def generate_param_todo_comment(
    param_name: str,  # Parameter name
    result: DocmentsCheckResult,  # Check result with type hint and doc info
    existing_comment: str = ""  # Existing comment text (without #)
) -> str:  # TODO comment to add
    "Generate appropriate TODO comment for a parameter based on what's missing"
    has_type_hint = result.params_with_type_hints.get(param_name, False)
    has_doc = result.params_documented.get(param_name, False)
    
    if not has_type_hint and not has_doc:
        # Missing both type hint and description
        return "TODO: Add type hint and description"
    elif not has_type_hint and has_doc:
        # Has description but missing type hint
        if existing_comment:
            # Check if TODO for type hint already exists
            if "TODO: Add type hint" in existing_comment or "TODO:Add type hint" in existing_comment:
                return existing_comment  # Don't add duplicate TODO
            else:
                return f"{existing_comment} - TODO: Add type hint"
        else:
            return "TODO: Add type hint"
    elif has_type_hint and not has_doc:
        # Has type hint but missing description
        return "TODO: Add description"
    else:
        # This shouldn't happen if we're being asked to generate a comment
        return existing_comment if existing_comment else "TODO: Verify documentation"

In [ ]:
#| export
def generate_return_todo_comment(
    result: DocmentsCheckResult,  # Check result with type hint and doc info
    existing_comment: str = ""  # Existing comment text (without #)
) -> str:  # TODO comment to add
    "Generate appropriate TODO comment for return value based on what's missing"
    has_type_hint = result.return_has_type_hint
    has_doc = result.return_documented
    
    if not has_type_hint and not has_doc:
        # Missing both type hint and description
        return "TODO: Add type hint and return description"
    elif not has_type_hint and has_doc:
        # Has description but missing type hint
        if existing_comment:
            # Check if TODO for type hint already exists
            if "TODO: Add type hint" in existing_comment or "TODO:Add type hint" in existing_comment:
                return existing_comment  # Don't add duplicate TODO
            else:
                return f"{existing_comment} - TODO: Add type hint"
        else:
            return "TODO: Add type hint"
    elif has_type_hint and not has_doc:
        # Has type hint but missing description
        return "TODO: Add return description"
    else:
        # This shouldn't happen if we're being asked to generate a comment
        return existing_comment if existing_comment else "TODO: Verify description"

In [ ]:
#| export
def build_fixed_single_line_function(
    parsed: dict,  # Parsed signature components
    params: List[str],  # Individual parameter strings
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Lines of fixed function signature
    "Build a fixed single-line function with documentation comments"
    fixed_lines = []
    indent = parsed['indent']
    
    # Start the function definition
    fixed_lines.append(f"{indent}{parsed['def_keyword']} {parsed['func_name']}(")
    
    # Add parameters with comments as needed
    for i, param in enumerate(params):
        # Use patch method to get parameter name
        param_name = result.get_param_name(param)
        
        # Use patch method to check if needs fixing
        if result.needs_param_fix(param_name):
            todo_comment = generate_param_todo_comment(param_name, result)
            if i < len(params) - 1:
                fixed_lines.append(f"{indent}    {param},  # {todo_comment}")
            else:
                fixed_lines.append(f"{indent}    {param}  # {todo_comment}")
        else:
            if i < len(params) - 1:
                fixed_lines.append(f"{indent}    {param},")
            else:
                fixed_lines.append(f"{indent}    {param}")
    
    # Handle return type and existing comment
    return_type = parsed['return_type']
    existing_comment = parsed['existing_comment']
    
    # Check if return type is None (no return value)
    is_none_return = return_type and 'None' in return_type.strip()
    
    # For single-line conversions, check if return needs fixing
    if return_type:
        # Skip adding TODO comments for functions with return type None
        if not is_none_return and ('return' in result.missing_params or 'return' in result.params_missing_type_hints):
            if existing_comment:
                # Parse existing comment
                comment_text = existing_comment[1:].strip() if existing_comment.startswith('#') else existing_comment
                todo_comment = generate_return_todo_comment(result, comment_text)
                fixed_lines.append(f"{indent}){return_type}: # {todo_comment}")
            else:
                # No existing comment
                todo_comment = generate_return_todo_comment(result)
                fixed_lines.append(f"{indent}){return_type}:  # {todo_comment}")
        else:
            # Return doesn't need fixing OR is None type
            if existing_comment:
                if existing_comment.startswith('#'):
                    fixed_lines.append(f"{indent}){return_type}: {existing_comment}")
                else:
                    fixed_lines.append(f"{indent}){return_type}: # {existing_comment}")
            else:
                fixed_lines.append(f"{indent}){return_type}:")
    else:
        # No return type but might need one
        if 'return' in result.params_missing_type_hints:
            if existing_comment:
                comment_text = existing_comment[1:].strip() if existing_comment.startswith('#') else existing_comment
                todo_comment = generate_return_todo_comment(result, comment_text)
                fixed_lines.append(f"{indent}): # {todo_comment}")
            else:
                todo_comment = generate_return_todo_comment(result)
                fixed_lines.append(f"{indent}): # {todo_comment}")
        else:
            # No return type needed
            if existing_comment:
                if existing_comment.startswith('#'):
                    fixed_lines.append(f"{indent}): {existing_comment}")
                else:
                    fixed_lines.append(f"{indent}): # {existing_comment}")
            else:
                fixed_lines.append(f"{indent}):")
    
    return fixed_lines

In [ ]:
#| export
def fix_multi_line_signature(
    lines: List[str],  # All source lines
    def_line_idx: int,  # Start of function definition
    sig_end_idx: int,  # End of function signature
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Fixed lines for the signature portion
    "Fix a multi-line function signature by adding parameter comments"
    fixed_lines = []
    
    for i in range(def_line_idx, sig_end_idx + 1):
        line = lines[i]
        line_stripped = line.strip()
        
        # More flexible parameter matching for multi-line signatures
        # Match: whitespace + word + optional type annotation + optional comma/paren + optional whitespace + optional comment
        param_match = re.match(r'^(\s*)(\w+)(\s*(?::\s*[^,\)#]+)?)\s*([,\)]?)(\s*)(?:#\s*(.*))?$', line)
        if param_match and i > def_line_idx and i < sig_end_idx:
            # This is a parameter line (not the def line, not the return line)
            indent = param_match.group(1)
            param_name = param_match.group(2)
            type_annotation = param_match.group(3) or ''
            trailing_punct = param_match.group(4) or ''
            trailing_space = param_match.group(5) or ''
            existing_comment = param_match.group(6) or ''
            
            # Check if this parameter needs fixing (either missing docs or missing type hints)
            needs_doc_fix = param_name in result.missing_params and param_name != 'self'
            needs_type_hint_fix = param_name in result.params_missing_type_hints and param_name != 'self'
            
            if needs_doc_fix or needs_type_hint_fix:
                todo_comment = generate_param_todo_comment(param_name, result, existing_comment)
                # Only add the fixed line if the comment actually changed
                if todo_comment != existing_comment:
                    fixed_lines.append(f"{indent}{param_name}{type_annotation}{trailing_punct}{trailing_space}  # {todo_comment}")
                else:
                    # Comment didn't change, keep original line
                    fixed_lines.append(line)
            else:
                fixed_lines.append(line)
        else:
            # Check for return type line
            return_match = re.match(r'^(\s*\)\s*->\s*[^:#]+)\s*:\s*(.*)$', line)
            if return_match:
                pre_colon = return_match.group(1)
                after_colon = return_match.group(2).strip()
                
                # Check if return type is None (no return value)
                is_none_return = 'None' in pre_colon
                
                # Skip adding TODO comments for functions with return type None
                if not is_none_return and ('return' in result.missing_params or 'return' in result.params_missing_type_hints):
                    if after_colon:
                        # There's already a comment, generate appropriate TODO
                        comment_text = after_colon[1:].strip() if after_colon.startswith('#') else after_colon
                        todo_comment = generate_return_todo_comment(result, comment_text)
                        # Only change if the comment actually changed
                        if todo_comment != comment_text:
                            fixed_lines.append(f"{pre_colon}: # {todo_comment}")
                        else:
                            fixed_lines.append(line)
                    else:
                        # No comment, add full TODO
                        todo_comment = generate_return_todo_comment(result)
                        fixed_lines.append(f"{pre_colon}:  # {todo_comment}")
                else:
                    fixed_lines.append(line)
            else:
                fixed_lines.append(line)
    
    return fixed_lines

In [ ]:
#| export
def fix_class_definition(
    result: DocmentsCheckResult  # Check result with non-compliant class
) -> str:  # Fixed source code with class docstring
    "Fix a class definition by adding a docstring if missing"
    lines = result.source.split('\n')
    fixed_lines = []
    
    # Find the class definition line
    class_line_idx = -1
    for i, line in enumerate(lines):
        if line.strip().startswith('class '):
            class_line_idx = i
            break
    
    if class_line_idx == -1:
        return result.source
    
    # Add lines up to and including the class definition
    for i in range(class_line_idx + 1):
        fixed_lines.append(lines[i])
    
    # If missing docstring, add it after the class definition
    if not result.has_docstring:
        # Find the indentation of the first line after class definition
        indent = '    '  # Default
        if class_line_idx + 1 < len(lines):
            next_line = lines[class_line_idx + 1]
            # Match leading whitespace
            indent_match = re.match(r'^(\s*)', next_line)
            indent = ifnone(indent_match.group(1) if indent_match else None, '    ')
        
        fixed_lines.append(f'{indent}"TODO: Add class description"')
    
    # Add the rest of the class body
    for i in range(class_line_idx + 1, len(lines)):
        fixed_lines.append(lines[i])
    
    return '\n'.join(fixed_lines)

In [ ]:
#| export
def insert_function_docstring(
    lines: List[str],  # Fixed function lines
    def_line_idx: int,  # Index of function definition line
    indent: str  # Base indentation for the function
) -> List[str]:  # Lines with docstring inserted
    "Insert a TODO docstring after the function signature"
    # Find the signature end (last line before function body)
    sig_end_idx = def_line_idx
    for i in range(def_line_idx, len(lines)):
        if lines[i].rstrip().endswith(':'):
            sig_end_idx = i
            break
    
    # Insert docstring after signature
    result_lines = []
    for i in range(sig_end_idx + 1):
        result_lines.append(lines[i])
    
    # Add the docstring
    docstring_indent = indent + '    '
    result_lines.append(f'{docstring_indent}"TODO: Add function description"')
    
    # Add the rest of the function body
    for i in range(sig_end_idx + 1, len(lines)):
        result_lines.append(lines[i])
    
    return result_lines

In [ ]:
#| export
def fix_single_line_function(
    lines: List[str],  # All source lines
    def_line_idx: int,  # Index of function definition line
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Fixed lines for the function
    "Fix a single-line function signature by converting to multi-line with parameter comments"
    # Parse the signature
    parsed = parse_single_line_signature(lines[def_line_idx])
    if not parsed:
        return lines
    
    # Split parameters
    params = split_parameters(parsed['params_str'])
    
    # Build the fixed function signature
    fixed_signature_lines = build_fixed_single_line_function(parsed, params, result)
    
    # Combine with rest of function
    fixed_lines = []
    # Add lines before the function
    for i in range(def_line_idx):
        fixed_lines.append(lines[i])
    
    # Add the fixed signature
    fixed_lines.extend(fixed_signature_lines)
    
    # Add docstring if missing
    if not result.has_docstring:
        docstring_indent = parsed['indent'] + '    '
        fixed_lines.append(f'{docstring_indent}"TODO: Add function description"')
    
    # Add lines after the function definition
    for i in range(def_line_idx + 1, len(lines)):
        fixed_lines.append(lines[i])
    
    return fixed_lines

In [ ]:
#| export
def fix_multi_line_function(
    lines: List[str],  # All source lines
    def_line_idx: int,  # Start of function definition
    sig_end_idx: int,  # End of function signature
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Fixed lines for the function
    "Fix a multi-line function signature by adding parameter comments"
    fixed_lines = []
    
    # Add lines before the function
    for i in range(def_line_idx):
        fixed_lines.append(lines[i])
    
    # Fix the signature
    signature_lines = fix_multi_line_signature(lines, def_line_idx, sig_end_idx, result)
    fixed_lines.extend(signature_lines)
    
    # Check if the function already has a docstring by looking at the first non-empty line after signature
    has_existing_docstring = False
    if sig_end_idx + 1 < len(lines):
        for i in range(sig_end_idx + 1, len(lines)):
            line_stripped = lines[i].strip()
            if line_stripped:  # First non-empty line
                # Check if it's a docstring
                if line_stripped.startswith(('"""', "'''", '"', "'")):
                    has_existing_docstring = True
                break
    
    # Insert docstring if missing AND not already present
    if not result.has_docstring and not has_existing_docstring:
        # Find the indentation of the function definition
        indent_match = re.match(r'^(\s*)', lines[def_line_idx])
        base_indent = indent_match.group(1) if indent_match else ''
        docstring_indent = base_indent + '    '
        fixed_lines.append(f'{docstring_indent}"TODO: Add function description"')
    
    # Add rest of function body
    for i in range(sig_end_idx + 1, len(lines)):
        fixed_lines.append(lines[i])
    
    return fixed_lines

In [ ]:
#| export
def generate_fixed_source(
    result: DocmentsCheckResult  # Check result with non-compliant function
) -> str:  # Fixed source code with placeholder documentation
    "Generate fixed source code for a non-compliant function or class"
    # Handle classes (including dataclasses)
    if result.type == 'ClassDef':
        return fix_class_definition(result)
    
    # Use the patch method to check if fixing is needed
    if not result.needs_fixing():
        return result.source
    
    lines = result.source.split('\n')
    
    # Find the function definition line and signature end
    def_line_idx, sig_end_idx = find_signature_boundaries(lines)
    
    if def_line_idx == -1:
        return result.source
    
    # Choose the appropriate fix method based on signature type
    if def_line_idx == sig_end_idx and (result.missing_params or result.params_missing_type_hints):
        # Single-line signature that needs parameter fixing
        fixed_lines = fix_single_line_function(lines, def_line_idx, result)
    else:
        # Multi-line signature 
        fixed_lines = fix_multi_line_function(lines, def_line_idx, sig_end_idx, result)
    
    return '\n'.join(fixed_lines)

In [ ]:
#| export
def fix_notebook(
    nb_path: Path,  # Path to notebook to fix
    dry_run: bool = False  # If True, show changes without saving
) -> Dict[str, Any]:  # Summary of changes made
    "Fix non-compliant functions in a notebook by adding placeholder documentation"
    nb = read_nb(nb_path)
    definitions = scan_notebook(nb_path)
    
    changes = {
        'notebook': nb_path.name,
        'definitions_fixed': [],
        'cells_modified': []
    }
    
    # Check each definition
    for defn in definitions:
        result = check_definition(defn)
        
        # Fix if non-compliant OR has missing type hints
        needs_fixing = (not result.is_compliant or 
                       result.missing_params or 
                       result.params_missing_type_hints)
        
        if needs_fixing:
            # Generate fixed source
            fixed_source = generate_fixed_source(result)
            
            # Only proceed if the source actually changed
            if fixed_source != result.source:
                # Find and update the cell
                cell_id = defn['cell_id']
                for cell in nb.cells:
                    if cell.get('id') == cell_id:
                        # Replace the definition in the cell source
                        old_source = result.source
                        cell_source = cell.source
                        
                        # Find the definition in the cell and replace it
                        if old_source in cell_source:
                            new_cell_source = cell_source.replace(old_source, fixed_source)
                            
                            if not dry_run:
                                cell.source = new_cell_source
                            
                            changes['definitions_fixed'].append(result.name)
                            if cell_id not in changes['cells_modified']:
                                changes['cells_modified'].append(cell_id)
                            
                            if dry_run:
                                print(f"\nWould fix {result.name}:")
                                print("-" * 40)
                                print(fixed_source)
                                print("-" * 40)
    
    # Save the notebook if not dry run
    if not dry_run and changes['definitions_fixed']:
        write_nb(nb, nb_path)
        # Fix grammar: use singular/plural based on count
        count = len(changes['definitions_fixed'])
        item_word = "definition" if count == 1 else "definitions"
        print(f"✅ Fixed {count} {item_word} in {nb_path.name}")
        for defn_name in changes['definitions_fixed']:
            print(f"   - {defn_name}")
    elif dry_run and changes['definitions_fixed']:
        count = len(changes['definitions_fixed'])
        item_word = "definition" if count == 1 else "definitions" 
        print(f"\n🔍 Dry run: Would fix {count} {item_word}")
    else:
        print(f"✅ All definitions in {nb_path.name} are already compliant")
    
    return changes

In [ ]:
#| export
class DocstringInfo(NamedTuple):
    """Information extracted from a docstring"""
    description: str  # Main function description
    params: Dict[str, str]  # Parameter name -> description
    returns: Optional[str]  # Return description
    docstring_type: str  # Type of docstring (google, numpy, sphinx, etc.)

In [ ]:
#| export
def detect_docstring_style(
    docstring: str  # Docstring text to analyze
) -> str:  # Detected style: 'google', 'numpy', 'sphinx', 'docments', or 'unknown'
    "Detect the style of a docstring"
    if not docstring:
        return 'unknown'
    
    docstring = docstring.strip()
    
    # Check for Google style (Args:, Returns:, etc.)
    if re.search(r'(Args?|Arguments?|Parameters?|Params?|Returns?|Return|Yields?|Yield|Raises?|Raise|Note|Notes|Example|Examples):\s*$', docstring, re.MULTILINE):
        return 'google'
    
    # Check for NumPy style (Parameters\n----------)
    if re.search(r'(Parameters?|Returns?|Yields?|Raises?|See Also|Notes?|References?|Examples?)\s*\n\s*-{3,}', docstring, re.MULTILINE):
        return 'numpy'
    
    # Check for Sphinx style (:param, :type, :returns, etc.)
    if re.search(r':(param|type|returns?|rtype|raises?|note|example)(\s+\w+)?:', docstring, re.MULTILINE):
        return 'sphinx'
    
    # Check if already in docments style (very simple check)
    # This would be harder to detect since docments puts docs inline
    # For now, assume unknown if none of the above patterns match
    return 'unknown'

In [ ]:
#| export
def parse_google_docstring(
    docstring: str  # Google-style docstring text
) -> DocstringInfo:  # Parsed docstring information
    "Parse a Google-style docstring"
    params = {}
    returns = None
    description_lines = []
    
    # Clean the docstring - remove triple quotes and normalize
    cleaned = docstring.strip()
    if cleaned.startswith('"""') or cleaned.startswith("'''"):
        cleaned = cleaned[3:]
    if cleaned.endswith('"""') or cleaned.endswith("'''"):
        cleaned = cleaned[:-3]
    
    lines = cleaned.split('\n')
    current_section = None
    current_param = None
    
    for line in lines:
        line = line.strip()
        
        # Check for section headers
        if re.match(r'^(Args?|Arguments?|Parameters?|Params?):\s*$', line):
            current_section = 'params'
            continue
        elif re.match(r'^(Returns?|Return):\s*$', line):
            current_section = 'returns'
            continue
        elif re.match(r'^(Yields?|Yield|Raises?|Raise|Note|Notes|Example|Examples):\s*$', line):
            current_section = 'other'
            continue
        
        # Process content based on current section
        if current_section == 'params':
            # Look for parameter definitions: "param_name (type): description"
            param_match = re.match(r'^(\w+)\s*(?:\([^)]+\))?\s*:\s*(.+)$', line)
            if param_match:
                param_name = param_match.group(1)
                param_desc = param_match.group(2)
                params[param_name] = param_desc
                current_param = param_name
            elif current_param and line:
                # Continuation of previous parameter description
                params[current_param] += ' ' + line
        elif current_section == 'returns':
            if line:
                if returns is None:
                    returns = line
                else:
                    returns += ' ' + line
        elif current_section is None:
            # This is part of the main description
            if line:
                description_lines.append(line)
    
    description = ' '.join(description_lines)
    return DocstringInfo(description, params, returns, 'google')

In [ ]:
#| export
def parse_numpy_docstring(
    docstring: str  # NumPy-style docstring text
) -> DocstringInfo:  # Parsed docstring information
    "Parse a NumPy-style docstring"
    params = {}
    returns = None
    description_lines = []
    
    # Clean the docstring - remove triple quotes and normalize
    cleaned = docstring.strip()
    if cleaned.startswith('"""') or cleaned.startswith("'''"):
        cleaned = cleaned[3:]
    if cleaned.endswith('"""') or cleaned.endswith("'''"):
        cleaned = cleaned[:-3]
    
    lines = cleaned.split('\n')
    current_section = None
    current_param = None
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        # Check for section headers (followed by dashes)
        if i + 1 < len(lines) and re.match(r'^-{3,}$', lines[i + 1].strip()):
            if re.match(r'^(Parameters?|Params?)$', line_stripped):
                current_section = 'params'
                continue
            elif re.match(r'^(Returns?|Return)$', line_stripped):
                current_section = 'returns'
                continue
            elif re.match(r'^(Yields?|Raises?|See Also|Notes?|References?|Examples?)$', line_stripped):
                current_section = 'other'
                continue
        
        # Skip the dashes line
        if re.match(r'^-{3,}$', line_stripped):
            continue
        
        # Process content based on current section
        if current_section == 'params':
            # Look for parameter definitions: "param_name : type" followed by description
            param_match = re.match(r'^(\w+)\s*:\s*(.+)$', line_stripped)
            if param_match:
                param_name = param_match.group(1)
                # The type information is on the same line, description usually follows
                current_param = param_name
                params[param_name] = ''
            elif current_param and line_stripped:
                # Description line for the current parameter
                if params[current_param]:
                    params[current_param] += ' ' + line_stripped
                else:
                    params[current_param] = line_stripped
        elif current_section == 'returns':
            if line_stripped:
                if returns is None:
                    returns = line_stripped
                else:
                    returns += ' ' + line_stripped
        elif current_section is None:
            # This is part of the main description
            if line_stripped:
                description_lines.append(line_stripped)
    
    description = ' '.join(description_lines)
    return DocstringInfo(description, params, returns, 'numpy')

In [ ]:
#| export
def parse_sphinx_docstring(
    docstring: str  # Sphinx-style docstring text
) -> DocstringInfo:  # Parsed docstring information
    "Parse a Sphinx-style docstring"
    params = {}
    returns = None
    description_lines = []
    
    lines = docstring.split('\n')
    
    for line in lines:
        line = line.strip()
        
        # Check for parameter definitions: ":param param_name: description"
        param_match = re.match(r'^:param\s+(\w+)\s*:\s*(.+)$', line)
        if param_match:
            param_name = param_match.group(1)
            param_desc = param_match.group(2)
            params[param_name] = param_desc
            continue
        
        # Check for return definitions: ":returns: description" or ":return: description"
        return_match = re.match(r'^:returns?\s*:\s*(.+)$', line)
        if return_match:
            returns = return_match.group(1)
            continue
        
        # Skip other sphinx directives
        if re.match(r'^:\w+(\s+\w+)?:', line):
            continue
        
        # This is part of the main description
        if line:
            description_lines.append(line)
    
    description = ' '.join(description_lines)
    return DocstringInfo(description, params, returns, 'sphinx')

In [ ]:
#| export
def extract_docstring_info(
    source: str,  # Function source code
    name: str  # Function name
) -> Optional[DocstringInfo]:  # Extracted docstring information or None
    "Extract docstring information from function source code"
    try:
        tree = ast.parse(source)
        for node in ast.walk(tree):
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                if node.name == name and node.body:
                    # Check if first statement is a docstring
                    first_stmt = node.body[0]
                    if (isinstance(first_stmt, ast.Expr) and 
                        isinstance(first_stmt.value, (ast.Str, ast.Constant))):
                        
                        # Extract docstring text
                        if hasattr(first_stmt.value, 's'):
                            docstring = first_stmt.value.s
                        elif hasattr(first_stmt.value, 'value'):
                            docstring = first_stmt.value.value
                        else:
                            return None
                        
                        if not isinstance(docstring, str):
                            return None
                        
                        # Detect and parse the docstring style
                        style = detect_docstring_style(docstring)
                        
                        if style == 'google':
                            return parse_google_docstring(docstring)
                        elif style == 'numpy':
                            return parse_numpy_docstring(docstring)
                        elif style == 'sphinx':
                            return parse_sphinx_docstring(docstring)
                        else:
                            # Unknown style, return basic info
                            return DocstringInfo(docstring.strip(), {}, None, 'unknown')
                    break
    except Exception:
        return None
    
    return None

In [ ]:
#| export
def convert_to_docments_format(
    source: str,  # Original function source code
    docstring_info: DocstringInfo,  # Extracted docstring information
    result: DocmentsCheckResult  # Check result with missing params info
) -> str:  # Converted source code in docments format
    "Convert function source to docments format using extracted docstring info"
    lines = source.split('\n')
    
    # Find the function definition line and signature end
    def_line_idx, sig_end_idx = find_signature_boundaries(lines)
    
    if def_line_idx == -1:
        return source
    
    # Build the new function with docments-style documentation
    fixed_lines = []
    
    # Add lines before the function
    for i in range(def_line_idx):
        fixed_lines.append(lines[i])
    
    # Convert single-line to multi-line if needed or fix existing multi-line
    if def_line_idx == sig_end_idx:
        # Single-line signature - convert to multi-line with docments comments
        fixed_lines.extend(convert_single_line_to_docments(lines[def_line_idx], docstring_info, result))
    else:
        # Multi-line signature - add docments comments to existing structure
        fixed_lines.extend(convert_multiline_to_docments(lines[def_line_idx:sig_end_idx+1], docstring_info, result))
    
    # Replace the original docstring with the description only
    body_start_idx = sig_end_idx + 1
    if body_start_idx < len(lines):
        # Find the docstring in the function body and replace it
        body_lines = lines[body_start_idx:]
        new_body_lines = replace_docstring_in_body(body_lines, docstring_info.description, lines[def_line_idx])
        fixed_lines.extend(new_body_lines)
    
    return '\n'.join(fixed_lines)

In [ ]:
#| export
def convert_single_line_to_docments(
    sig_line: str,  # Single-line function signature
    docstring_info: DocstringInfo,  # Extracted docstring information
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Multi-line signature with docments comments
    "Convert single-line function signature to multi-line docments format"
    
    # Parse the signature
    parsed = parse_single_line_signature(sig_line)
    if not parsed:
        return [sig_line]
    
    # Split parameters
    params = split_parameters(parsed['params_str'])
    
    # Build the new signature
    fixed_lines = []
    indent = parsed['indent']
    
    # Start the function definition
    fixed_lines.append(f"{indent}{parsed['def_keyword']} {parsed['func_name']}(")
    
    # Add parameters with docments comments
    for i, param in enumerate(params):
        param_name = result.get_param_name(param)
        
        # Get documentation from the extracted docstring info
        param_doc = docstring_info.params.get(param_name, '')
        
        if param_doc:
            # Use the extracted documentation
            if i < len(params) - 1:
                fixed_lines.append(f"{indent}    {param},  # {param_doc}")
            else:
                fixed_lines.append(f"{indent}    {param}  # {param_doc}")
        else:
            # No documentation found, add TODO
            if param_name in result.missing_params:
                todo_comment = generate_param_todo_comment(param_name, result)
                if i < len(params) - 1:
                    fixed_lines.append(f"{indent}    {param},  # {todo_comment}")
                else:
                    fixed_lines.append(f"{indent}    {param}  # {todo_comment}")
            else:
                # Keep as is
                if i < len(params) - 1:
                    fixed_lines.append(f"{indent}    {param},")
                else:
                    fixed_lines.append(f"{indent}    {param}")
    
    # Handle return type
    return_type = parsed['return_type']
    
    # Check if return type is None (no return value)
    is_none_return = return_type and 'None' in return_type.strip()
    
    if return_type and docstring_info.returns and not is_none_return:
        fixed_lines.append(f"{indent}){return_type}:  # {docstring_info.returns}")
    elif return_type and 'return' in result.missing_params and not is_none_return:
        todo_comment = generate_return_todo_comment(result)
        fixed_lines.append(f"{indent}){return_type}:  # {todo_comment}")
    elif return_type:
        fixed_lines.append(f"{indent}){return_type}:")
    else:
        fixed_lines.append(f"{indent}):")
    
    return fixed_lines

In [ ]:
#| export
def convert_multiline_to_docments(
    sig_lines: List[str],  # Multi-line function signature
    docstring_info: DocstringInfo,  # Extracted docstring information
    result: DocmentsCheckResult  # Check result with missing params info
) -> List[str]:  # Multi-line signature with docments comments
    "Convert multi-line function signature to docments format"
    
    fixed_lines = []
    
    for i, line in enumerate(sig_lines):
        line_stripped = line.strip()
        
        # Check if this line contains a parameter (including the last one that might end with ))
        # Updated regex to handle parameters that end with )) for the last param before return type
        param_match = re.match(r'^(\s*)(\w+)(\s*(?::\s*[^,\)#]+)?)\s*([,\)]?)(\)?)\s*(?:->\s*[^:#]+)?\s*:?\s*(?:#\s*(.*))?$', line)
        
        # Check if it's a parameter line (not the def line)
        if param_match and i > 0 and param_match.group(2) != 'def':
            param_name = param_match.group(2)
            
            # Skip if this is actually the return type line
            if '->' in line and ')' in line:
                # This is a return type line, handle it separately
                return_match = re.match(r'^(\s*.*\)\s*->\s*[^:#]+)\s*:\s*(.*)$', line)
                if return_match:
                    pre_colon = return_match.group(1)
                    existing_comment = return_match.group(2).strip()
                    
                    # Check if return type is None (no return value)
                    is_none_return = 'None' in pre_colon
                    
                    # Check if the existing comment is a TODO comment
                    is_todo_comment = 'TODO' in existing_comment if existing_comment else False
                    
                    if docstring_info.returns and not is_none_return and (not existing_comment or is_todo_comment):
                        # Replace with extracted return documentation if no comment or it's a TODO
                        fixed_lines.append(f"{pre_colon}:  # {docstring_info.returns}")
                    elif 'return' in result.missing_params and not is_none_return:
                        comment_text = existing_comment[1:].strip() if existing_comment.startswith('#') else existing_comment
                        todo_comment = generate_return_todo_comment(result, comment_text)
                        fixed_lines.append(f"{pre_colon}:  # {todo_comment}")
                    else:
                        fixed_lines.append(line)
                else:
                    fixed_lines.append(line)
            else:
                # Regular parameter line
                indent = param_match.group(1)
                type_annotation = param_match.group(3) or ''
                trailing_punct = param_match.group(4) or ''
                extra_paren = param_match.group(5) or ''
                existing_comment = param_match.group(6) or ''
                
                # Reconstruct the line ending (could be , or ) or ))
                line_ending = trailing_punct + extra_paren
                
                # Get documentation from the extracted docstring info
                param_doc = docstring_info.params.get(param_name, '')
                
                # Check if the existing comment is a TODO comment
                is_todo_comment = 'TODO' in existing_comment
                
                if param_doc and (not existing_comment or is_todo_comment):
                    # Replace with the extracted documentation if there's no comment or it's a TODO
                    fixed_lines.append(f"{indent}{param_name}{type_annotation}{line_ending}  # {param_doc}")
                elif param_doc and existing_comment and not is_todo_comment:
                    # Keep existing non-TODO comment (it might be manually written documentation)
                    fixed_lines.append(line)
                elif param_name in result.missing_params:
                    # No documentation found in docstring, add TODO
                    todo_comment = generate_param_todo_comment(param_name, result, existing_comment)
                    fixed_lines.append(f"{indent}{param_name}{type_annotation}{line_ending}  # {todo_comment}")
                else:
                    # Keep original
                    fixed_lines.append(line)
        else:
            # Not a parameter line, could be def line or other
            fixed_lines.append(line)
    
    return fixed_lines

In [ ]:
#| export
def replace_docstring_in_body(
    body_lines: List[str],  # Function body lines
    description: str,  # New description to use
    def_line: str  # Function definition line for indentation
) -> List[str]:  # Modified body lines
    "Replace the docstring in function body with a simple description"
    
    # Find the indentation of the function definition
    indent_match = re.match(r'^(\s*)', def_line)
    base_indent = indent_match.group(1) if indent_match else ''
    docstring_indent = base_indent + '    '
    
    # Look for the docstring (first string literal after function definition)
    docstring_found = False
    result_lines = []
    in_multiline_docstring = False
    
    for i, line in enumerate(body_lines):
        line_stripped = line.strip()
        
        # If we haven't found the docstring yet and this line is not empty
        if not docstring_found and line_stripped:
            # Check if it starts a docstring
            if line_stripped.startswith(('"""', "'''", '"', "'")):
                docstring_found = True
                
                # Check if it's a single-line docstring
                if ((line_stripped.startswith('"""') and line_stripped.endswith('"""') and len(line_stripped) > 6) or
                    (line_stripped.startswith("'''") and line_stripped.endswith("'''") and len(line_stripped) > 6) or
                    (line_stripped.startswith('"') and line_stripped.endswith('"') and len(line_stripped) > 2 and not line_stripped.startswith('"""')) or
                    (line_stripped.startswith("'") and line_stripped.endswith("'") and len(line_stripped) > 2 and not line_stripped.startswith("'''"))):
                    # Single-line docstring
                    result_lines.append(f'{docstring_indent}"{description}"')
                else:
                    # Start of multi-line docstring
                    in_multiline_docstring = True
                    result_lines.append(f'{docstring_indent}"{description}"')
            else:
                # Not a docstring, keep the line
                result_lines.append(line)
        elif in_multiline_docstring:
            # We're inside a multi-line docstring, check if this ends it
            if line_stripped.endswith(('"""', "'''")):
                in_multiline_docstring = False
                # Skip this line (end of docstring)
            # Skip all lines inside the multi-line docstring
        else:
            # Either we already processed the docstring or this is a regular line
            result_lines.append(line)
    
    # If no docstring was found, add the description at the beginning
    if not docstring_found:
        result_lines.insert(0, f'{docstring_indent}"{description}"')
    
    return result_lines

In [ ]:
#| export
def generate_fixed_source_with_conversion(
    result: DocmentsCheckResult  # Check result with non-compliant function
) -> str:  # Fixed source code with converted documentation
    "Generate fixed source code, converting existing docstrings to docments format if possible"
    
    # First, try to extract docstring information for conversion
    docstring_info = extract_docstring_info(result.source, result.name)
    
    # If we found structured docstring info (not unknown), convert it
    if (docstring_info and 
        docstring_info.docstring_type in ['google', 'numpy', 'sphinx'] and
        (docstring_info.params or docstring_info.returns)):
        try:
            converted_source = convert_to_docments_format(result.source, docstring_info, result)
            return converted_source
        except Exception:
            # Fallback to original fix if conversion fails
            pass
    
    # Fallback to the original generate_fixed_source function
    return generate_fixed_source(result)

In [ ]:
#| export
def fix_notebook_with_conversion(
    nb_path: Path,  # Path to notebook to fix
    dry_run: bool = False,  # If True, show changes without saving
    convert_docstrings: bool = True  # If True, convert existing docstrings to docments format
) -> Dict[str, Any]:  # Summary of changes made
    "Fix non-compliant functions in a notebook, optionally converting docstrings to docments format"
    nb = read_nb(nb_path)
    definitions = scan_notebook(nb_path)
    
    changes = {
        'notebook': nb_path.name,
        'definitions_fixed': [],
        'definitions_converted': [],
        'cells_modified': []
    }
    
    # Check each definition
    for defn in definitions:
        result = check_definition(defn)
        
        # Fix if non-compliant OR has missing type hints
        needs_fixing = (not result.is_compliant or 
                       result.missing_params or 
                       result.params_missing_type_hints)
        
        if needs_fixing:
            # Choose the appropriate fix method
            if convert_docstrings:
                fixed_source = generate_fixed_source_with_conversion(result)
                
                # Check if this was a conversion (has structured docstring info)
                docstring_info = extract_docstring_info(result.source, result.name)
                is_conversion = (docstring_info and 
                               docstring_info.docstring_type in ['google', 'numpy', 'sphinx'])
            else:
                fixed_source = generate_fixed_source(result)
                is_conversion = False
            
            # Only proceed if the source actually changed
            if fixed_source != result.source:
                # Find and update the cell
                cell_id = defn['cell_id']
                for cell in nb.cells:
                    if cell.get('id') == cell_id:
                        # Replace the definition in the cell source
                        old_source = result.source
                        cell_source = cell.source
                        
                        # Find the definition in the cell and replace it
                        if old_source in cell_source:
                            new_cell_source = cell_source.replace(old_source, fixed_source)
                            
                            if not dry_run:
                                cell.source = new_cell_source
                            
                            changes['definitions_fixed'].append(result.name)
                            if is_conversion:
                                changes['definitions_converted'].append(result.name)
                            
                            if cell_id not in changes['cells_modified']:
                                changes['cells_modified'].append(cell_id)
                            
                            if dry_run:
                                action = "convert and fix" if is_conversion else "fix"
                                print(f"\nWould {action} {result.name}:")
                                print("-" * 40)
                                print(fixed_source)
                                print("-" * 40)
    
    # Save the notebook if not dry run
    if not dry_run and changes['definitions_fixed']:
        write_nb(nb, nb_path)
        
        # Report results
        fixed_count = len(changes['definitions_fixed'])
        converted_count = len(changes['definitions_converted'])
        
        if converted_count > 0:
            print(f"✅ Fixed {fixed_count} definitions in {nb_path.name} ({converted_count} converted from other docstring styles)")
        else:
            print(f"✅ Fixed {fixed_count} definitions in {nb_path.name}")
        
        for defn_name in changes['definitions_fixed']:
            action = "converted & fixed" if defn_name in changes['definitions_converted'] else "fixed"
            print(f"   - {defn_name} ({action})")
    elif dry_run and changes['definitions_fixed']:
        fixed_count = len(changes['definitions_fixed'])
        converted_count = len(changes['definitions_converted'])
        
        if converted_count > 0:
            print(f"\n🔍 Dry run: Would fix {fixed_count} definitions ({converted_count} converted from other docstring styles)")
        else:
            print(f"\n🔍 Dry run: Would fix {fixed_count} definitions")
    else:
        print(f"✅ All definitions in {nb_path.name} are already compliant")
    
    return changes

### Testing

In [ ]:
def test_docstring_detection_and_parsing():
    """Test docstring style detection and parsing for all supported formats"""
    print("🧪 Testing Docstring Detection and Parsing")
    print("=" * 50)
    
    # Test docstrings for different styles
    test_docstrings = [
        # Google style
        ('google', '''"""Calculate the sum of two numbers.
        
        Args:
            x (int): The first number to add
            y (int): The second number to add
            
        Returns:
            int: The sum of x and y
        """'''),
        
        # NumPy style  
        ('numpy', '''"""Calculate the sum of two numbers.
        
        Parameters
        ----------
        x : int
            The first number to add
        y : int  
            The second number to add
            
        Returns
        -------
        int
            The sum of x and y
        """'''),
        
        # Sphinx style
        ('sphinx', '''"""Calculate the sum of two numbers.
        
        :param x: The first number to add
        :param y: The second number to add
        :returns: The sum of x and y
        """'''),
        
        # Unknown style
        ('unknown', '''"""Just a simple description without structured parameters."""''')
    ]
    
    # Test detection
    print("📋 Style Detection Results:")
    for expected_style, docstring in test_docstrings:
        detected_style = detect_docstring_style(docstring)
        status = "✅" if detected_style == expected_style else "❌"
        print(f"{status} {expected_style.title()}: {detected_style}")
    
    # Test parsing for structured formats
    print("\n📖 Parsing Results:")
    for style_name, docstring in test_docstrings[:3]:  # Skip unknown style
        if style_name == 'google':
            parsed = parse_google_docstring(docstring)
        elif style_name == 'numpy':
            parsed = parse_numpy_docstring(docstring)
        elif style_name == 'sphinx':
            parsed = parse_sphinx_docstring(docstring)
        
        print(f"\n{style_name.title()} parsing:")
        print(f"  Description: {parsed.description}")
        print(f"  Parameters: {list(parsed.params.keys())}")
        print(f"  Returns: {'Yes' if parsed.returns else 'No'}")
    
    print("\n✅ Docstring detection and parsing tests completed")

# Run test
test_docstring_detection_and_parsing()

🧪 Testing Docstring Detection and Parsing
📋 Style Detection Results:
✅ Google: google
✅ Numpy: numpy
✅ Sphinx: sphinx
✅ Unknown: unknown

📖 Parsing Results:

Google parsing:
  Description: Calculate the sum of two numbers.
  Parameters: ['x', 'y']
  Returns: Yes

Numpy parsing:
  Description: Calculate the sum of two numbers.
  Parameters: ['x', 'y']
  Returns: Yes

Sphinx parsing:
  Description: """Calculate the sum of two numbers. """
  Parameters: ['x', 'y']
  Returns: Yes

✅ Docstring detection and parsing tests completed


In [ ]:
def test_function_fixing():
    """Test basic function fixing for various scenarios"""
    print("\n🔧 Testing Function Fixing")
    print("=" * 50)
    
    # Test cases with different compliance issues
    test_cases = [
        {
            'name': 'missing_all_docs',
            'source': '''def bad_function(x, y, z=10):
    result = x + y + z
    return result''',
            'args': [
                {'name': 'x', 'annotation': None},
                {'name': 'y', 'annotation': None},
                {'name': 'z', 'annotation': None}
            ],
            'returns': None,
            'description': 'Missing all documentation and type hints'
        },
        {
            'name': 'typed_function',
            'source': '''def typed_function(name: str, age: int) -> str:
    return f"{name} is {age} years old"''',
            'args': [
                {'name': 'name', 'annotation': 'str'},
                {'name': 'age', 'annotation': 'int'}
            ],
            'returns': 'str',
            'description': 'Has type hints but missing parameter documentation'
        },
        {
            'name': 'partially_documented',
            'source': '''def get_export_cells(
    nb_path: Path,  # Path to the notebook file
    fake_test_path: Path 
) -> List[Dict[str, Any]]:  # List of cells with export directives
    "Extract all code cells from a notebook that have export directives"
    nb = read_nb(nb_path)
    return []''',
            'args': [
                {'name': 'nb_path', 'annotation': 'Path'},
                {'name': 'fake_test_path', 'annotation': 'Path'}
            ],
            'returns': 'List[Dict[str, Any]]',
            'description': 'Partially documented - missing one parameter doc'
        }
    ]
    
    for test_case in test_cases:
        print(f"\n📝 Testing: {test_case['description']}")
        print(f"Function: {test_case['name']}")

        print(f"\nSource:\n{test_case['source']}\n")
        
        # Create test definition
        test_def = {
            'name': test_case['name'],
            'type': 'FunctionDef',
            'source': test_case['source'],
            'notebook': 'test.ipynb',
            'args': test_case['args'],
            'returns': test_case['returns']
        }
        
        # Check compliance
        result = check_definition(test_def)
        print(f"Has docstring: {result.has_docstring}")
        print(f"Is compliant: {result.is_compliant}")
        if not result.is_compliant:
            print(f"Missing: {result.missing_params}")
            
            # Apply fix
            fixed_source = generate_fixed_source(result)
            print(f"\nFixed Source:\n{fixed_source}\n")
    
    print("\n✅ Function fixing tests completed")

# Run test
test_function_fixing()


🔧 Testing Function Fixing

📝 Testing: Missing all documentation and type hints
Function: missing_all_docs

Source:
def bad_function(x, y, z=10):
    result = x + y + z
    return result

Has docstring: False
Is compliant: False
Missing: ['x', 'y', 'z']

Fixed Source:
def bad_function(
    x,  # TODO: Add type hint and description
    y,  # TODO: Add type hint and description
    z=10  # TODO: Add type hint and description
):
    "TODO: Add function description"
    result = x + y + z
    return result


📝 Testing: Has type hints but missing parameter documentation
Function: typed_function

Source:
def typed_function(name: str, age: int) -> str:
    return f"{name} is {age} years old"

Has docstring: False
Is compliant: False
Missing: ['name', 'age', 'return']

Fixed Source:
def typed_function(
    name: str,  # TODO: Add description
    age: int  # TODO: Add description
) -> str:  # TODO: Add return description
    "TODO: Add function description"
    return f"{name} is {age} years ol

In [ ]:
def test_docstring_conversion():
    """Test conversion from various docstring formats to docments style"""
    print("\n🔄 Testing Docstring Conversion")
    print("=" * 50)
    
    # Test functions with different docstring formats
    test_functions = [
        {
            'name': 'google_example',
            'source': '''def google_example(name: str, age: int, active: bool = True) -> str:
    """Generate a user profile string.
    
    Args:
        name (str): The user's full name
        age (int): The user's age in years
        active (bool): Whether the user is currently active
        
    Returns:
        str: A formatted profile string
    """
    return f"{name} ({age}) - {'Active' if active else 'Inactive'}"''',
            'args': [
                {'name': 'name', 'annotation': 'str'},
                {'name': 'age', 'annotation': 'int'},
                {'name': 'active', 'annotation': 'bool'}
            ],
            'returns': 'str',
            'style': 'Google'
        },
        {
            'name': 'numpy_example',
            'source': '''def numpy_example(data: list, threshold: float = 0.5) -> dict:
    """Process data based on threshold.
    
    Parameters
    ----------
    data : list
        Input data to process
    threshold : float
        Minimum threshold value
        
    Returns
    -------
    dict
        Processing results with statistics
    """
    return {'processed': len(data), 'threshold': threshold}''',
            'args': [
                {'name': 'data', 'annotation': 'list'},
                {'name': 'threshold', 'annotation': 'float'}
            ],
            'returns': 'dict',
            'style': 'NumPy'
        }
    ]
    
    for func_info in test_functions:
        print(f"\n📝 Testing {func_info['style']} Style Conversion")
        print(f"Function: {func_info['name']}")

        print(f"\nSource:\n{func_info['source']}\n")
        
        # Create test definition
        test_def = {
            'name': func_info['name'],
            'type': 'FunctionDef',
            'source': func_info['source'],
            'notebook': 'test.ipynb',
            'args': func_info['args'],
            'returns': func_info['returns']
        }
        
        # Check original compliance
        result = check_definition(test_def)
        
        
        # Extract and verify docstring info
        docstring_info = extract_docstring_info(result.source, result.name)
        if docstring_info:
            print(f"Docstring type: {docstring_info.docstring_type}")
            print(f"Parameters found: {list(docstring_info.params.keys())}")
            print(f"Return info: {'Yes' if docstring_info.returns else 'No'}")
        
        # Convert to docments format
        converted = generate_fixed_source_with_conversion(result)
        print(f"\nConverted Source:\n{converted}\n")
        
        # Verify converted version is compliant
        test_def_converted = test_def.copy()
        test_def_converted['source'] = converted
        result_converted = check_definition(test_def_converted)
    
    print("\n✅ Docstring conversion tests completed")

# Run test
test_docstring_conversion()


🔄 Testing Docstring Conversion

📝 Testing Google Style Conversion
Function: google_example

Source:
def google_example(name: str, age: int, active: bool = True) -> str:
    """Generate a user profile string.

    Args:
        name (str): The user's full name
        age (int): The user's age in years
        active (bool): Whether the user is currently active

    Returns:
        str: A formatted profile string
    """
    return f"{name} ({age}) - {'Active' if active else 'Inactive'}"

Docstring type: google
Parameters found: ['name', 'age', 'active']
Return info: Yes

Converted Source:
def google_example(
    name: str,  # The user's full name
    age: int,  # The user's age in years
    active: bool = True  # Whether the user is currently active
) -> str:  # str: A formatted profile string
    "Generate a user profile string."
    return f"{name} ({age}) - {'Active' if active else 'Inactive'}"


📝 Testing NumPy Style Conversion
Function: numpy_example

Source:
def numpy_example(d

In [ ]:
def test_edge_cases():
    """Test edge cases and special scenarios"""
    print("\n🎯 Testing Edge Cases")
    print("=" * 50)
    
    # Test dataclass without docstring
    dataclass_source = '''@dataclass
class DocmentsCheckResult:
    name: str  # Name of the function/class
    type: str  # Type (FunctionDef, ClassDef, etc.)
    notebook: str  # Source notebook
    has_docstring: bool  # Whether it has a docstring
    params_documented: Dict[str, bool]  # Which params have documentation
    return_documented: bool  # Whether return is documented
    missing_params: List[str]  # Parameters missing documentation
    is_compliant: bool  # Overall compliance status'''

    print(f"\nDataClass Source:{dataclass_source}\n")    
    
    test_dataclass = {
        'name': 'DocmentsCheckResult',
        'type': 'ClassDef',
        'source': dataclass_source,
        'notebook': 'test.ipynb',
        'args': [],
        'returns': None
    }
    
    print("📝 Testing dataclass without docstring")
    result = check_definition(test_dataclass)
    
    if not result.is_compliant:
        fixed = generate_fixed_source(result)
        print(f"\nFixed DataClass Source:\n{fixed}\n")
    
    # Test single-line function with existing return comment
    edge_case_source = '''
def get_export_cells(nb_path: Path) -> List[Dict[str, Any]]:  # List of cells with export directives
    "Extract all code cells from a notebook that have export directives"
    nb = read_nb(nb_path)
    return []'''

    print(f"\nEdge Case Source:{edge_case_source}\n")
    
    test_edge_case = {
        'name': 'get_export_cells',
        'type': 'FunctionDef',
        'source': edge_case_source,
        'notebook': 'test.ipynb',
        'args': [{'name': 'nb_path', 'annotation': 'Path'}],
        'returns': 'List[Dict[str, Any]]'
    }
    
    print("\n📝 Testing single-line function with existing return comment")
    result_edge = check_definition(test_edge_case)
    
    if not result_edge.is_compliant:
        fixed_edge = generate_fixed_source(result_edge)
        print(f"\nFixed Edge Case Source:\n{fixed_edge}\n")
        test_edge_fixed = test_edge_case.copy()
        test_edge_fixed['source'] = fixed_edge
        result_edge_after = check_definition(test_edge_fixed)

# Run test
test_edge_cases()


🎯 Testing Edge Cases

DataClass Source:@dataclass
class DocmentsCheckResult:
    name: str  # Name of the function/class
    type: str  # Type (FunctionDef, ClassDef, etc.)
    notebook: str  # Source notebook
    has_docstring: bool  # Whether it has a docstring
    params_documented: Dict[str, bool]  # Which params have documentation
    return_documented: bool  # Whether return is documented
    missing_params: List[str]  # Parameters missing documentation
    is_compliant: bool  # Overall compliance status

📝 Testing dataclass without docstring

Fixed DataClass Source:
@dataclass
class DocmentsCheckResult:
    "TODO: Add class description"
    name: str  # Name of the function/class
    type: str  # Type (FunctionDef, ClassDef, etc.)
    notebook: str  # Source notebook
    has_docstring: bool  # Whether it has a docstring
    params_documented: Dict[str, bool]  # Which params have documentation
    return_documented: bool  # Whether return is documented
    missing_params: List[s

In [ ]:
# Test fixing functions without return values
def test_no_return_value_functions():
    """Test that functions without return values don't get TODO: Add type hint comments"""
    print("\n🔍 Testing Functions Without Return Values")
    print("=" * 50)
    
    from cjm_nbdev_docments.core import function_has_return_value
    
    test_cases = [
        {
            'name': 'output_report',
            'source': '''def output_report(
    report: str,  # Report content to output
    output_path: Optional[Path] = None,  # File path to save report to
    quiet: bool = False  # Whether to suppress output
):  # TODO: Add return description
    "Output the report to console or file"
    if output_path:
        output_path.write_text(report)
        if not quiet:
            print(f"Report saved to {output_path}")
    elif not quiet:
        print(report)''',
            'args': [
                {'name': 'report', 'annotation': 'str'},
                {'name': 'output_path', 'annotation': 'Optional[Path]'},
                {'name': 'quiet', 'annotation': 'bool'}
            ],
            'returns': None,
            'description': 'Function with no return value'
        },
        {
            'name': 'print_only',
            'source': '''def print_only(x, y):
    "Just print values"
    print(f"{x} and {y}")''',
            'args': [
                {'name': 'x', 'annotation': None},
                {'name': 'y', 'annotation': None}
            ],
            'returns': None,
            'description': 'Simple function with no return'
        },
        {
            'name': 'with_return',
            'source': '''def with_return(x, y):
    "Add and return"
    return x + y''',
            'args': [
                {'name': 'x', 'annotation': None},
                {'name': 'y', 'annotation': None}
            ],
            'returns': None,
            'description': 'Function that returns a value'
        }
    ]
    
    for test_case in test_cases:
        print(f"\n📝 Testing: {test_case['description']}")
        print(f"Function: {test_case['name']}")
        
        # Check if function has return value
        has_return = function_has_return_value(test_case['source'], test_case['name'])
        print(f"Has return value: {has_return}")
        
        # Create test definition
        test_def = {
            'name': test_case['name'],
            'type': 'FunctionDef',
            'source': test_case['source'],
            'notebook': 'test.ipynb',
            'args': test_case['args'],
            'returns': test_case['returns']
        }
        
        # Check compliance
        result = check_definition(test_def)
        print(f"Missing type hints: {result.params_missing_type_hints}")
        print(f"'return' in missing type hints: {'return' in result.params_missing_type_hints}")
        
        # Apply fix if needed
        if not result.is_compliant or result.params_missing_type_hints:
            fixed_source = generate_fixed_source(result)
            print(f"\nFixed Source:\n{fixed_source}\n")
            
            # Verify no TODO for return type hint was added if no return value
            if not has_return and "TODO: Add type hint" in fixed_source and "):" in fixed_source:
                # Check if TODO is on the return line
                for line in fixed_source.split('\n'):
                    if ')' in line and ':' in line and 'TODO: Add type hint' in line and 'def' not in line:
                        print("❌ ERROR: Added TODO for return type hint when function has no return value!")
                        break
    
    print("\n✅ No-return-value function tests completed")

# Run test
test_no_return_value_functions()


🔍 Testing Functions Without Return Values

📝 Testing: Function with no return value
Function: output_report
Has return value: False
Missing type hints: []
'return' in missing type hints: False

📝 Testing: Simple function with no return
Function: print_only
Has return value: False
Missing type hints: ['x', 'y']
'return' in missing type hints: False

Fixed Source:
def print_only(
    x,  # TODO: Add type hint and description
    y  # TODO: Add type hint and description
):
    "Just print values"
    print(f"{x} and {y}")


📝 Testing: Function that returns a value
Function: with_return
Has return value: True
Missing type hints: ['x', 'y', 'return']
'return' in missing type hints: True

Fixed Source:
def with_return(
    x,  # TODO: Add type hint and description
    y  # TODO: Add type hint and description
): # TODO: Add type hint
    "Add and return"
    return x + y


✅ No-return-value function tests completed


In [ ]:
def test_simplified_docstring_format():
    """Test the simplified docstring format (Args without type info in parentheses)"""
    print("\n🆕 Testing Simplified Docstring Format")
    print("=" * 50)
    
    # The exact docstring format the user wants to support
    test_docstring = '''"""Add an element with OOB swap configuration.

Args:
    element: The element to add
    target_id: Target element ID for OOB swap
    swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
    wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
          If False, add OOB attributes directly to the element.

Returns:
    Self for chaining
"""'''
    
    print("Testing docstring:")
    print(test_docstring)
    print()
    
    # Test detection
    detected_style = detect_docstring_style(test_docstring)
    print(f"Detected style: {detected_style}")
    
    # Test parsing - should be detected as Google style
    if detected_style == 'google':
        parsed = parse_google_docstring(test_docstring)
        print(f"✅ Successfully detected as Google style")
        print(f"Description: {parsed.description}")
        print(f"Parameters parsed: {list(parsed.params.keys())}")
        print(f"Parameter details:")
        for param, desc in parsed.params.items():
            print(f"  - {param}: {desc}")
        print(f"Returns: {parsed.returns}")
    else:
        print(f"❌ Not detected as Google style (detected as {detected_style})")
    
    # Test conversion with a complete function
    test_function_source = '''def add_oob_element(element, target_id, swap_mode, wrap=True):
    """Add an element with OOB swap configuration.

    Args:
        element: The element to add
        target_id: Target element ID for OOB swap
        swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
        wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
              If False, add OOB attributes directly to the element.

    Returns:
        Self for chaining
    """
    # Implementation here
    return self'''
    
    print("\n📝 Testing conversion of simplified format:")
    
    # Create test definition
    test_def = {
        'name': 'add_oob_element',
        'type': 'FunctionDef',
        'source': test_function_source,
        'notebook': 'test.ipynb',
        'args': [
            {'name': 'element', 'annotation': None},
            {'name': 'target_id', 'annotation': None},
            {'name': 'swap_mode', 'annotation': None},
            {'name': 'wrap', 'annotation': None}
        ],
        'returns': None
    }
    
    # Check compliance
    result = check_definition(test_def)
    print(f"Is compliant before conversion: {result.is_compliant}")
    print(f"Missing params: {result.missing_params}")
    
    # Extract docstring info
    docstring_info = extract_docstring_info(test_function_source, 'add_oob_element')
    if docstring_info:
        print(f"\nExtracted docstring info:")
        print(f"  Style: {docstring_info.docstring_type}")
        print(f"  Params: {list(docstring_info.params.keys())}")
        
        # Convert to docments format
        converted = generate_fixed_source_with_conversion(result)
        print(f"\nConverted to docments format:")
        print(converted)
    
    print("\n✅ Simplified docstring format test completed")

# Run test
test_simplified_docstring_format()


🆕 Testing Simplified Docstring Format
Testing docstring:
"""Add an element with OOB swap configuration.

Args:
    element: The element to add
    target_id: Target element ID for OOB swap
    swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
    wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
          If False, add OOB attributes directly to the element.

Returns:
    Self for chaining
"""

Detected style: google
✅ Successfully detected as Google style
Description: Add an element with OOB swap configuration.
Parameters parsed: ['element', 'target_id', 'swap_mode', 'wrap']
Parameter details:
  - element: The element to add
  - target_id: Target element ID for OOB swap
  - swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
  - wrap: If True and target_id is provided, wrap content in a Div with OOB attributes. If False, add OOB attributes directly to the element.
Returns: Self for chaining

📝 Testing conve

In [ ]:
def test_mixed_documentation_scenario():
    """Test converting docstring when parameters already have TODO comments"""
    print("\n🔄 Testing Mixed Documentation Scenario")
    print("=" * 50)
    
    # This is the exact scenario from the user's code
    test_source = '''def add_element(self,
                   element: Any,  # TODO: Add description
                   target_id: Optional[str] = None,  # TODO: Add description
                   swap_mode: str = "innerHTML",  # TODO: Add description
                   wrap: bool = True) -> 'OOBStreamBuilder':
        """Add an element with OOB swap configuration.
        
        Args:
            element: The element to add
            target_id: Target element ID for OOB swap
            swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
            wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
                  If False, add OOB attributes directly to the element.
            
        Returns:
            Self for chaining
        """
        return self'''
    
    print("Original source with TODOs and docstring:")
    print(test_source)
    print()
    
    # Create test definition
    test_def = {
        'name': 'add_element',
        'type': 'FunctionDef',
        'source': test_source,
        'notebook': 'test.ipynb',
        'args': [
            {'name': 'self', 'annotation': None},
            {'name': 'element', 'annotation': 'Any'},
            {'name': 'target_id', 'annotation': 'Optional[str]'},
            {'name': 'swap_mode', 'annotation': 'str'},
            {'name': 'wrap', 'annotation': 'bool'}
        ],
        'returns': "'OOBStreamBuilder'"
    }
    
    # Check compliance
    result = check_definition(test_def)
    print(f"Is compliant: {result.is_compliant}")
    print(f"Missing params: {result.missing_params}")
    print(f"Has TODOs: {result.has_todos}")
    print()
    
    # Extract docstring info
    docstring_info = extract_docstring_info(test_source, 'add_element')
    if docstring_info:
        print(f"Extracted docstring info:")
        print(f"  Style: {docstring_info.docstring_type}")
        print(f"  Params: {list(docstring_info.params.keys())}")
        print()
    
    # Try to convert
    converted = generate_fixed_source_with_conversion(result)
    print("After conversion attempt:")
    print(converted)
    
    print("\n✅ Mixed documentation scenario test completed")

# Run test
test_mixed_documentation_scenario()


🔄 Testing Mixed Documentation Scenario
Original source with TODOs and docstring:
def add_element(self,
                   element: Any,  # TODO: Add description
                   target_id: Optional[str] = None,  # TODO: Add description
                   swap_mode: str = "innerHTML",  # TODO: Add description
                   wrap: bool = True) -> 'OOBStreamBuilder':
        """Add an element with OOB swap configuration.

        Args:
            element: The element to add
            target_id: Target element ID for OOB swap
            swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
            wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
                  If False, add OOB attributes directly to the element.

        Returns:
            Self for chaining
        """
        return self

Is compliant: False
Missing params: ['wrap', 'return']
Has TODOs: True

Extracted docstring info:
  Style: google
  Params: ['el

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()